# YouTube Video URL Fetcher

This script demonstrates how to use the YouTube Data API to fetch video URLs from a specific YouTube channel. The script performs the following tasks:

1. Retrieves the channel ID for a given channel name.
2. Retrieves the playlist ID for the uploads playlist of the channel.
3. Fetches all video IDs from the playlist.
4. Converts video IDs into YouTube video URLs.
5. Saves the list of video URLs to a specified file.

Place your YouTube API key in a `.env` file as shown below:

```dotenv
YOUTUBE_API_KEY=your_youtube_api_key_here


In [1]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build

# Load environment variables from a .env file
load_dotenv()  # Loads .env file to get API keys.

# API Key and channel name
api_key = os.getenv("YOUTUBE_API_KEY")  # Retrieve the YouTube API key from environment variables
channel_name = 'TelekomHilfe-Videos'  # The name of the YouTube channel to search for

def get_channel_id(api_key, channel_name):
    """
    Retrieves the channel ID for a given channel name using the YouTube Data API.
    
    Args:
        api_key (str): The API key for authenticating with the YouTube Data API.
        channel_name (str): The name of the YouTube channel to search for.
        
    Returns:
        str or None: The channel ID if found, otherwise None.
    """
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    # Search for the channel by name
    request = youtube.search().list(
        part='snippet',
        q=channel_name,
        type='channel',
        maxResults=1
    )
    response = request.execute()
    
    # Extract and return the channel ID if available
    if response['items']:
        return response['items'][0]['id']['channelId']
    else:
        return None

def get_playlist_id(api_key, channel_id):
    """
    Retrieves the playlist ID for the uploads playlist of a given channel ID.
    
    Args:
        api_key (str): The API key for authenticating with the YouTube Data API.
        channel_id (str): The ID of the YouTube channel.
        
    Returns:
        str or None: The playlist ID for uploads if found, otherwise None.
    """
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    # Get the channel details, including the uploads playlist ID
    request = youtube.channels().list(
        part='contentDetails',
        id=channel_id
    )
    response = request.execute()
    
    # Extract and return the playlist ID if available
    if response['items']:
        playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        return playlist_id
    else:
        return None

def get_video_ids_from_playlist(api_key, playlist_id):
    """
    Retrieves all video IDs from a given playlist.
    
    Args:
        api_key (str): The API key for authenticating with the YouTube Data API.
        playlist_id (str): The ID of the playlist to fetch video IDs from.
        
    Returns:
        list of str: A list of video IDs from the playlist.
    """
    youtube = build('youtube', 'v3', developerKey=api_key)
    video_ids = []
    
    # Request video IDs from the playlist
    request = youtube.playlistItems().list(
        part='contentDetails',
        playlistId=playlist_id,
        maxResults=50
    )
    
    while request:
        response = request.execute()
        # Append video IDs to the list
        video_ids.extend(item['contentDetails']['videoId'] for item in response.get('items', []))
        # Get the next page of results if available
        request = youtube.playlistItems().list_next(request, response)
    
    return video_ids

def get_video_urls(video_ids):
    """
    Converts a list of video IDs into YouTube video URLs.
    
    Args:
        video_ids (list of str): A list of video IDs to convert.
        
    Returns:
        list of str: A list of full YouTube video URLs.
    """
    return [f"https://www.youtube.com/watch?v={video_id}" for video_id in video_ids]

def save_urls_to_file(urls, file_path):
    """
    Saves a list of video URLs to a specified file.
    
    Args:
        urls (list of str): The list of URLs to save.
        file_path (str): The path to the file where URLs will be saved.
    """
    os.makedirs(os.path.dirname(file_path), exist_ok=True)  # Ensure the directory exists
    with open(file_path, 'w') as file:
        for url in urls:
            file.write(url + '\n')  # Write each URL on a new line

def main():
    """
    Main function to execute the process of fetching video URLs from a YouTube channel
    and saving them to a file.
    """
    # Retrieve the channel ID for the given channel name
    channel_id = get_channel_id(api_key, channel_name)
    if channel_id:
        # Retrieve the playlist ID for the uploads playlist of the channel
        playlist_id = get_playlist_id(api_key, channel_id)
        if playlist_id:
            # Get video IDs from the playlist
            video_ids = get_video_ids_from_playlist(api_key, playlist_id)
            
            # Convert video IDs to URLs
            video_urls = get_video_urls(video_ids)
            
            # Save the video URLs to a file
            output_file = 'data/video_urls.txt'
            save_urls_to_file(video_urls, output_file)
            print(f"Fetched {len(video_urls)} video URLs and saved to {output_file}")
        else:
            print("Playlist not found.")
    else:
        print("Channel not found.")

if __name__ == "__main__":
    main()


Fetched 243 video URLs and saved to data/video_urls.txt


# Downloading and Processing Audio Files from YouTube

This notebook automates the process of extracting audio from YouTube videos and converting them to MP3 format, making it useful for collecting and processing audio content for various purposes.


### Libraries Used

- **`os`**: Provides a way to interact with the operating system, including file and directory operations.
- **`yt_dlp`**: A command-line program to download videos from YouTube and other video platforms, used here to extract audio from videos.
- **`re`**: Supports regular expression operations to manipulate and validate filenames.

### Code Workflow

1. **Read Video URLs**:
   - Video URLs are read from `data/video_urls.txt`, with each URL stripped of extra whitespace.

2. **Configure `yt-dlp`**:
   - Set up options for `yt-dlp` to download the best available audio format and convert it to MP3 format.
   - The output files are saved in the directory `data/mp3_files/`.

3. **Download and Process Audio**:
   - For each URL, the script extracts video information and downloads the audio.
   - The audio file is then saved as an MP3 file using a sanitized filename.
   - If an error occurs during download, it is caught and reported.
   - After downloading, the script verifies the presence of the MP3 file and prints a confirmation message.

4. **Completion**:
   - A message is printed to indicate that all processes have been completed.

### Error Handling

- Errors during the download process are caught and printed, allowing the script to continue processing other URLs even if one fails.
- The existence of the MP3 file is checked to confirm successful download.

In [2]:
import os
import yt_dlp
import re

# Function to create a safe filename by removing or replacing invalid characters
def safe_filename(filename):
    # Remove invalid characters for filenames
    filename = re.sub(r'[\\/*?:"<>|]', "", filename)
    # Replace spaces with underscores
    filename = re.sub(r'\s+', '_', filename)
    # Limit the filename length to 255 characters
    filename = filename[:255]
    return filename.lower()

# Read video URLs from 'data/video_urls.txt'
input_file_path = "data/video_urls.txt"
with open(input_file_path, "r") as file:
    video_urls = [line.strip() for line in file.readlines()]

# yt-dlp options for downloading and processing audio files
ydl_opts = {
    # Download the best available audio format
    'format': 'bestaudio/best',
    # Post-process the downloaded file to extract audio and convert to MP3
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    # Output template for the downloaded files
    'outtmpl': 'data/mp3_files/%(title)s.%(ext)s',
    # Suppress verbose output
    'quiet': True,
}

# Loop through each video URL to download and save audio files
for video_url in video_urls:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            # Extract information and download the video
            info_dict = ydl.extract_info(video_url, download=True)
            # Get the title of the video
            title = info_dict.get('title', None)
            # Create a safe filename from the video title
            safe_title = safe_filename(title)
            # Define the path for the MP3 file
            mp3_filename = f"data/mp3_files/{safe_title}.mp3"
        except Exception as e:
            # Print an error message if there is an issue with downloading
            print(f"Download error: {e}")
            continue

        # Check if the MP3 file has been successfully created
        if not os.path.exists(mp3_filename):
            print(f"File not found: {mp3_filename}")
            continue

        # Print a success message with the path of the downloaded MP3 file
        print(f"MP3 file successfully downloaded: {mp3_filename}")

# Indicate that all processes have completed
print("All operations completed.")


File not found: data/mp3_files/magentatv_statt_kabel-tv_-_was_der_wegfall_des_nebenkostenprivilegs_für_dich_bedeutet!.mp3
File not found: data/mp3_files/kuriose_kundennachrichten_…_#shorts_#telekom_#magenta_#comedy_#jokes_#funny_#fyp_#like_#follow.mp3
File not found: data/mp3_files/werde_teil_der_telekom_hilft_community!_#telekom_#shorts_#magenta_#fyp_#community_#like_#follow_#yt.mp3
File not found: data/mp3_files/frag_magenta_der_digitale_assistent_#shorts#telekom#fragmagenta#support#tech#digitalart#fyp#like.mp3
File not found: data/mp3_files/wlan-name_ändern_-_fritzbox_und_speedport!_#telekom_#magenta_#wlan_#tech_#fyp_#support_#help_#change.mp3
File not found: data/mp3_files/erste_hilfe_bei_internetstörungen_#telekom_#magenta_#wlan_#problem_#help_#fyp_#tech_#support_#like.mp3
File not found: data/mp3_files/meinmagenta_app_halte_dein_datenvolumen_im_blick!_#shorts_#magenta_#telekom_#diy_#fyp_#technology.mp3
File not found: data/mp3_files/beste_wlan-verbindung_bei_dir_zuhause!_#telekom

# MP3 Files Transcription

It is designed to handle the transcription of MP3 audio files using the Whisper model. The notebook performs the following steps:

1. **Conversion of MP3 Files to WAV Format**:
   - MP3 files are converted to WAV format as Whisper works more efficiently with WAV files for transcription.

2. **Transcription of WAV Files**:
   - The converted WAV files are processed using the Whisper model to generate transcriptions.

3. **Saving Transcriptions**:
   - The transcriptions are saved as text files in a specified directory.

In [3]:
# transcribe_mp3_files.ipynb

import os
import whisper
from pydub import AudioSegment

# Define paths for input and output folders
mp3_folder = 'data/mp3_files'  # Folder containing the MP3 files to be processed
wav_folder = 'data/wav_files'  # Folder where converted WAV files will be saved
transcript_folder = 'data/transcripts'  # Folder where transcriptions will be saved

# Create necessary folders if they do not exist
os.makedirs(wav_folder, exist_ok=True)  # Create the WAV files folder if it does not exist
os.makedirs(transcript_folder, exist_ok=True)  # Create the transcripts folder if it does not exist

# Load the Whisper model for transcription
model = whisper.load_model("base")

# Function to convert MP3 file to WAV format
def convert_mp3_to_wav(mp3_path, wav_path):
    """
    Convert an MP3 file to WAV format using the AudioSegment module.
    
    Parameters:
    mp3_path (str): Path to the input MP3 file.
    wav_path (str): Path where the output WAV file will be saved.
    """
    audio = AudioSegment.from_mp3(mp3_path)  # Load MP3 file
    audio.export(wav_path, format='wav')  # Export as WAV file

# Function to transcribe audio file in WAV format
def transcribe_audio(wav_path):
    """
    Transcribe the audio from a WAV file using the Whisper model.
    
    Parameters:
    wav_path (str): Path to the WAV file to be transcribed.
    
    Returns:
    str: Transcribed text from the audio file.
    """
    result = model.transcribe(wav_path)  # Transcribe the WAV file
    return result['text']  # Return the transcribed text

# Process each MP3 file in the specified folder
for mp3_file in os.listdir(mp3_folder):
    if mp3_file.endswith('.mp3'):  # Check if the file has an MP3 extension
        mp3_path = os.path.join(mp3_folder, mp3_file)  # Full path to the MP3 file
        wav_file = mp3_file.replace('.mp3', '.wav')  # Generate the WAV filename
        wav_path = os.path.join(wav_folder, wav_file)  # Full path to the WAV file
        transcript_file = mp3_file.replace('.mp3', '.txt')  # Generate the transcript filename
        transcript_path = os.path.join(transcript_folder, transcript_file)  # Full path to the transcript file
        
        # Convert MP3 file to WAV format
        convert_mp3_to_wav(mp3_path, wav_path)
        
        # Transcribe the WAV file
        transcript = transcribe_audio(wav_path)
        
        # Save the transcription to a text file
        with open(transcript_path, 'w') as f:
            f.write(transcript)
        
        # Print a message indicating that the transcription has been saved
        print(f"Transcription saved for: {mp3_file}")


/Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=dev

Transcription saved for: Änderung der LAN IP Adresse bei der Digitalisierungsbox Smart.mp3


/Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription saved for: Для українців： як перевести безкоштовну картку мобільного телефону на передплачений тариф I Telekom.mp3


/Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

Transcription saved for: 6 WLAN-Tipps gegen Störenfriede! #shorts #telekom #magenta #hilfe #wlan #störung #signal.mp3


/Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription saved for: Digitalisierungsbox Premium - Schnelle Einrichtung eines SIP-Trunks im Media Gateway Modus.mp3


/Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription saved for: Telekom： MagentaTV statt Kabel-TV - Wegfall Nebenkostenprivileg.mp3


# Youtube QA generated via gpt-4
## 'data/transcripts.' kaynagi önemli

In [1]:
import os
import glob
from openai import OpenAI
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Setup OpenAI API key and client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Define paths
transcripts_dir = 'data/transcripts'
organized_data_dir = "/Users/taha/Desktop/rag/data"
if not os.path.exists(organized_data_dir):
    os.makedirs(organized_data_dir)

# Define categories
categories = [
    "Geräte & Zubehör",
    "Hilfe bei Störungen",
    "Internet & Telefonie",
    "MagentaEINS",
    "Mobilfunk",
    "TV",
    "Vertrag & Rechnung",
    "Apps & Dienste",
    "Others"
]

# Function to categorize transcript, summarize, and generate questions
def process_transcript(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            transcript = file.read()

        # Request GPT-4 for category
        category_response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Please categorize the following transcript into one of the categories: {', '.join(categories)}:\n\n{transcript}. Please provide only the chosen category name."}
            ],
            max_tokens=20,
        )
        category = category_response.choices[0].message.content.strip()

        # Ensure the category is valid
        if category not in categories:
            category = "Others"

        # Request GPT-4 for summary
        summary_response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful telecom help assistant."},
                {"role": "user", "content": f"Please summarize the following transcript in a concise and clear manner in German. This summary will be used as webpage content:\n\n{transcript}"}
            ],
        )
        summary = summary_response.choices[0].message.content.strip()

        # Request GPT-4 for generating a question
        question_response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful telecom help assistant."},
                {"role": "user", "content": f"Based on the following transcript, generate a relevant question in German:\n\n{transcript}"}
            ],
            max_tokens=50,
        )
        question = question_response.choices[0].message.content.strip()

        return category, summary, question, transcript
    except Exception as e:
        logging.error(f"Error processing {file_path}: {e}")
        return "Unknown", None, None, None

# Process all transcript files
for transcript_file in glob.glob(os.path.join(transcripts_dir, '*.txt')):
    filename = os.path.basename(transcript_file)
    title, _ = os.path.splitext(filename)
    
    category, summary, question, transcript = process_transcript(transcript_file)
    if summary and question:
        # Create category directory if it doesn't exist
        category_dir = os.path.join(organized_data_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

        # Define output path with 'youtube_' prefix
        output_path = os.path.join(category_dir, f'youtube_{title}.txt')
        with open(output_path, 'w', encoding='utf-8') as combined_file:
            combined_file.write("Question:\n")
            combined_file.write(question + "\n\n")
            combined_file.write("Answer:\n")
            combined_file.write(summary + "\n\n")

            # Transcripti txt dosyasina eklemek icin bunu ekle koda
            #combined_file.write("Youtube transcript:\n")
            #combined_file.write(transcript)
        logging.info(f"Processed {filename} and saved to {output_path}")
    else:
        logging.warning(f"Skipping {filename} due to errors.")

print("Processing complete. Combined files saved in their respective category folder.")

2024-09-01 22:45:31,260 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-01 22:45:44,725 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-01 22:45:45,743 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-01 22:45:45,748 - INFO - Processed WLAN-Namen ändern： So geht's beim Speedport ｜ Telekom.txt and saved to /Users/taha/Desktop/rag/data/Internet & Telefonie/youtube_WLAN-Namen ändern： So geht's beim Speedport ｜ Telekom.txt
2024-09-01 22:45:46,590 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-01 22:45:57,314 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-01 22:45:59,055 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-01 22:45:59,059 - INFO - Processed Telekom： Digitalisierungsbox Smart 2 & Premium 2 

Processing complete. Combined files saved in their respective category folders under 'website/organized_data'.
